In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as data
import yfinance as yf

ModuleNotFoundError: No module named 'distutils'

In [ ]:
start = '2020-01-01'
end = '2025-05-09'

# Get the data
df = yf.download('RELIANCE.NS', start=start, end=end)
df = df[['Open', 'High', 'Low', 'Close']]
df.reset_index(inplace=True)
df.set_index('Date', inplace=True)
# print(df.head())
df.head()

In [ ]:
df.tail()

In [ ]:
# plt.plot(df.Close)

In [ ]:
df

In [ ]:
ma100 = df.Close.rolling(100).mean()
ma100

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df.Close)
plt.plot(ma100, 'r')

In [ ]:
ma200 = df.Close.rolling(200).mean()
ma200

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df.Close)
plt.plot(ma100, 'r')
plt.plot(ma200, 'g')
plt.legend(['Close', 'MA100', 'MA200'])
plt.title('Stock Price with 100 and 200 days moving average')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

In [ ]:
df.shape

In [ ]:
# splitting the data into training and test set

data_training = pd.DataFrame(df['Close'][0:int(len(df)*0.70)])
data_testing = pd.DataFrame(df['Close'][int(len(df)*0.70):int(len(df))])

print(data_training.shape)
print(data_testing.shape)

In [ ]:
data_training.head()

In [ ]:
data_testing.head()

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
data_training_array = scaler.fit_transform(data_training)
data_training_array

In [17]:
x_train = []
y_train = []

for i in range(100, data_training_array.shape[0]):
    x_train.append(data_training_array[i-100:i])
    y_train.append(data_training_array[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [18]:
#ML model

from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential
from keras.optimizers import Adam
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping

In [19]:
# model = Sequential()
# model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1))) 
# model.add(Dropout(0.2))

# model.add(LSTM(units=60, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1))) 
# model.add(Dropout(0.3))

# model.add(LSTM(units=80, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1))) 
# model.add(Dropout(0.4))

# model.add(LSTM(units=120, activation='relu')) 
# model.add(Dropout(0.5))

# model.add(Dense(units=1))


model = Sequential()

# First LSTM layer
model.add(LSTM(units=128, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))

# Second LSTM layer
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))

# Third LSTM layer
model.add(LSTM(units=32))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(units=1))

In [ ]:
model.summary()

In [ ]:

# model.compile(optimizer='adam', loss='mean_squared_error')
# model.fit(x_train, y_train, epochs = 100) #Train the model using the training data

# Compile the model
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mean_squared_error')
# Fit the model to the training data
model.fit(x_train, y_train, epochs=100, batch_size=32)

In [ ]:
# model.save('./LSTM_model.h5')
model.save('./LSTM_model_adam_new.h5')

In [ ]:
data_testing.head()

In [24]:
past_100_days = data_training.tail(100)

In [ ]:
final_df = past_100_days.append(data_testing, ignore_index=True)

In [ ]:
final_df.head()

In [ ]:
input_data = scaler.fit_transform(final_df)
input_data

In [ ]:
input_data.shape

In [29]:
x_test = []
y_test = []

for i in range(100, input_data.shape[0]):
    x_test.append(input_data[i-100:i])
    y_test.append(input_data[i, 0])

In [ ]:
x_test, y_test = np.array(x_test), np.array(y_test)
print(x_test.shape)
print(y_test.shape)

In [ ]:
#making predictions
y_pred = model.predict(x_test)

In [ ]:
y_pred.shape

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
scaler.scale_

In [36]:
# scale_factor = 1/0.00101725
scale_factor = 1/scaler.scale_
y_pred = y_pred*scale_factor
y_test = y_test*scale_factor

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test, 'b', label='Original Price')
plt.plot(y_pred, 'r', label='Predicted Price')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.title('Stock Price Prediction')
plt.legend()
plt.show()

# Improving the LSTM Model Prediction

Several strategies can be implemented to improve the model's prediction accuracy:

1. **Feature Engineering**: Add more relevant features beyond just the closing price
2. **Hyperparameter Tuning**: Optimize model parameters
3. **Model Architecture**: Experiment with different LSTM architectures
4. **Regularization**: Apply techniques to prevent overfitting
5. **Evaluation Metrics**: Use proper metrics to evaluate model performance

## 1. Feature Engineering - Adding Technical Indicators

In [ ]:
# Create a new DataFrame with additional features
def add_features(df):
    df_features = df.copy()
    
    # 1. Moving Averages
    df_features['MA7'] = df['Close'].rolling(window=7).mean()
    df_features['MA21'] = df['Close'].rolling(window=21).mean()
    
    # 2. Standard Deviation (Volatility)
    df_features['STD20'] = df['Close'].rolling(window=20).std()
    
    # 3. Relative Strength Index (RSI)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df_features['RSI'] = 100 - (100 / (1 + rs))
    
    # 4. MACD (Moving Average Convergence Divergence)
    exp1 = df['Close'].ewm(span=12, adjust=False).mean()
    exp2 = df['Close'].ewm(span=26, adjust=False).mean()
    df_features['MACD'] = exp1 - exp2
    df_features['Signal_Line'] = df_features['MACD'].ewm(span=9, adjust=False).mean()
    
    # 5. Bollinger Bands
    df_features['BB_Upper'] = df_features['MA21'] + (df_features['STD20'] * 2)
    df_features['BB_Lower'] = df_features['MA21'] - (df_features['STD20'] * 2)
    
    # 6. Price Rate of Change
    df_features['ROC'] = df['Close'].pct_change(periods=5) * 100
    
    # Forward fill NaN values
    df_features = df_features.fillna(method='ffill')
    # Backward fill any remaining NaN values
    df_features = df_features.fillna(method='bfill')
    
    return df_features

# Apply feature engineering
enhanced_df = add_features(df)
enhanced_df.head()

## 2. Hyperparameter Tuning with Keras Tuner

In [ ]:
# Install keras-tuner if not already installed
# !pip install -q keras-tuner

import keras_tuner as kt
from tensorflow import keras

def build_model(hp):
    model = Sequential()
    
    # Tune the number of units in the first LSTM layer
    hp_units_lstm1 = hp.Int('units_lstm1', min_value=32, max_value=256, step=32)
    model.add(LSTM(units=hp_units_lstm1, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)))
    
    # Tune the number of units in the second LSTM layer
    hp_units_lstm2 = hp.Int('units_lstm2', min_value=16, max_value=128, step=16))
    model.add(LSTM(units=hp_units_lstm2, return_sequences=True))
    model.add(Dropout(hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)))
    
    # Tune the number of units in the third LSTM layer
    hp_units_lstm3 = hp.Int('units_lstm3', min_value=8, max_value=64, step=8))
    model.add(LSTM(units=hp_units_lstm3))
    model.add(Dropout(hp.Float('dropout3', min_value=0.0, max_value=0.5, step=0.1)))
    
    # Output layer
    model.add(Dense(units=1))
    
    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-4, 1e-3, 1e-2])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                 loss='mean_squared_error')
    
    return model

# Create the tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=50,
    factor=3,
    directory='keras_tuner',
    project_name='lstm_stock_prediction'
)

In [ ]:
# Create validation data from the training set
val_split = 0.2  # 20% of training data for validation
val_idx = int(x_train.shape[0] * (1 - val_split))

x_t = x_train[:val_idx]
y_t = y_train[:val_idx]
x_val = x_train[val_idx:]
y_val = y_train[val_idx:]

# Define early stopping callback
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Search for the best hyperparameters
tuner.search(x_t, y_t,
            epochs=50,
            validation_data=(x_val, y_val),
            callbacks=[early_stopping],
            verbose=1)

# Get the best model
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best hyperparameters:\n{best_hp.values}")
best_model = tuner.hypermodel.build(best_hp)

## 3. Enhanced Training with Multiple Features

In [ ]:
# Select the features we want to use for prediction
feature_columns = ['Close', 'MA7', 'MA21', 'RSI', 'MACD', 'ROC']
features_df = enhanced_df[feature_columns]

# Split data into training and testing sets
train_data = features_df.iloc[0:int(len(features_df)*0.70)]
test_data = features_df.iloc[int(len(features_df)*0.70):]

# Scale the data
multi_feature_scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_data = multi_feature_scaler.fit_transform(train_data)

# Prepare the training sequences for multivariate input
lookback = 100  # Same lookback period as before
x_multi_train = []
y_multi_train = []

for i in range(lookback, scaled_train_data.shape[0]):
    x_multi_train.append(scaled_train_data[i-lookback:i])
    # We still predict the Close price (which is the first column)
    y_multi_train.append(scaled_train_data[i, 0])

x_multi_train, y_multi_train = np.array(x_multi_train), np.array(y_multi_train)

print(f"Multivariate training data shape: {x_multi_train.shape}")

In [ ]:
# Build an improved model using best hyperparameters but adapted for multivariate input
multi_input_model = Sequential()

# First LSTM layer
multi_input_model.add(LSTM(units=best_hp.get('units_lstm1'), 
                           return_sequences=True, 
                           input_shape=(lookback, len(feature_columns))))
multi_input_model.add(Dropout(best_hp.get('dropout1')))

# Second LSTM layer
multi_input_model.add(LSTM(units=best_hp.get('units_lstm2'), return_sequences=True))
multi_input_model.add(Dropout(best_hp.get('dropout2')))

# Third LSTM layer
multi_input_model.add(LSTM(units=best_hp.get('units_lstm3')))
multi_input_model.add(Dropout(best_hp.get('dropout3')))

# Output layer
multi_input_model.add(Dense(units=1))

# Compile the model
multi_input_model.compile(optimizer=Adam(learning_rate=best_hp.get('learning_rate')), 
                         loss='mean_squared_error')

multi_input_model.summary()

In [ ]:
# Train the multivariate model
history = multi_input_model.fit(
    x_multi_train, 
    y_multi_train, 
    epochs=100, 
    batch_size=32,
    validation_split=0.2,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
    verbose=1
)

## 4. Evaluating the Enhanced Model

In [ ]:
# Prepare the test data for multivariate prediction
# Scale the test data
scaled_test_data = multi_feature_scaler.transform(test_data)

# Prepare the test sequences
x_multi_test = []
y_multi_test = []

for i in range(lookback, len(scaled_test_data)):
    x_multi_test.append(scaled_test_data[i-lookback:i])
    y_multi_test.append(scaled_test_data[i, 0])  # First column is Close price

x_multi_test, y_multi_test = np.array(x_multi_test), np.array(y_multi_test)

# Make predictions
y_multi_pred = multi_input_model.predict(x_multi_test)

# Create a dummy array to inverse transform the predictions
multi_pred_inverse = np.zeros((len(y_multi_pred), len(feature_columns)))
multi_test_inverse = np.zeros((len(y_multi_test), len(feature_columns)))

# Place predictions and actual values in the first column (Close price)
multi_pred_inverse[:, 0] = y_multi_pred.flatten()
multi_test_inverse[:, 0] = y_multi_test.flatten()

# Inverse transform to get the actual stock prices
y_multi_pred_actual = multi_feature_scaler.inverse_transform(multi_pred_inverse)[:, 0]
y_multi_test_actual = multi_feature_scaler.inverse_transform(multi_test_inverse)[:, 0]

In [ ]:
# Import evaluation metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate error metrics for both models
def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"{model_name} Model Performance:")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
    print(f"R-squared (R²): {r2:.4f}")
    print("\n")
    
    return mse, rmse, mae, mape, r2

# Evaluate both models
print("Comparing Model Performance:\n")
original_metrics = evaluate_model(y_test, y_pred, "Original")
enhanced_metrics = evaluate_model(y_multi_test_actual, y_multi_pred_actual, "Enhanced")

In [ ]:
# Save the enhanced model
multi_input_model.save('./LSTM_model_enhanced.h5')

## 5. Visualizing Results with Advanced Plotting

In [ ]:
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
import seaborn as sns

# Get original dates for better visualization
test_dates = test_data.index

# Set the style for better visualizations
sns.set_style('whitegrid')
plt.figure(figsize=(16, 10))

# Plot actual vs predicted prices
plt.subplot(2, 1, 1)
plt.plot(test_dates[-len(y_multi_test_actual):], y_multi_test_actual, 'b-', linewidth=2, label='Actual Price')
plt.plot(test_dates[-len(y_multi_pred_actual):], y_multi_pred_actual, 'r--', linewidth=2, label='Enhanced Model Prediction')
plt.title('Stock Price Prediction - Enhanced Model', fontsize=16)
plt.ylabel('Price (INR)', fontsize=14)
plt.legend(loc='best', fontsize=12)
plt.grid(True, alpha=0.3)

# Plot prediction error
plt.subplot(2, 1, 2)
error = y_multi_test_actual - y_multi_pred_actual
plt.plot(test_dates[-len(error):], error, 'g-', label='Prediction Error')
plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)
plt.fill_between(test_dates[-len(error):], error, 0, alpha=0.3, color='g' if np.mean(error) < 0 else 'r')
plt.title('Prediction Error', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Error (INR)', fontsize=14)
plt.legend(loc='best', fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Forecasting Future Prices

In [ ]:
# Function to forecast future stock prices
def forecast_future_prices(model, last_sequence, n_steps, scaler, n_features):
    """
    Forecast future stock prices
    
    Parameters:
    model: Trained LSTM model
    last_sequence: Last observed sequence of features used for prediction
    n_steps: Number of future steps to predict
    scaler: Fitted MinMaxScaler used for feature scaling
    n_features: Number of features in the input
    
    Returns:
    Array of predicted prices
    """
    future_predictions = []
    current_sequence = last_sequence.copy()
    
    for _ in range(n_steps):
        # Reshape for model input
        current_reshape = current_sequence.reshape(1, current_sequence.shape[0], current_sequence.shape[1])
        # Predict next value
        next_pred = model.predict(current_reshape)
        
        # Create a sequence for inverse transform
        dummy = np.zeros((1, n_features))
        dummy[0, 0] = next_pred[0, 0]
        next_price = scaler.inverse_transform(dummy)[0, 0]
        future_predictions.append(next_price)
        
        # Update sequence for next prediction
        # For simplicity, we only update the close price
        # In a real application, you would need to generate new values for all features
        dummy_feature_row = np.zeros((1, n_features))
        dummy_feature_row[0, 0] = next_pred[0, 0]  # Close price
        
        # For simplicity, copying the last values of other features
        # This is a simplified approach; ideally, you'd calculate new values
        current_sequence = np.append(current_sequence[1:], [dummy_feature_row[0]], axis=0)
    
    return np.array(future_predictions)

In [ ]:
# Get the last sequence from our test data
last_sequence = x_multi_test[-1]

# Forecast next 30 days
forecast_days = 30
forecasted_prices = forecast_future_prices(
    multi_input_model, 
    last_sequence, 
    forecast_days, 
    multi_feature_scaler, 
    len(feature_columns)
)

# Generate future dates for plotting
last_date = test_dates[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=forecast_days, freq='B')

# Plot the historical data along with the forecasted prices
plt.figure(figsize=(14, 7))

# Plot historical data
historical_dates = test_dates[-60:]  # Last 60 days of historical data
historical_prices = y_multi_test_actual[-60:]
plt.plot(historical_dates, historical_prices, 'b-', label='Historical Prices')

# Plot forecasted data
plt.plot(future_dates, forecasted_prices, 'r--', label='Forecasted Prices')

# Add a vertical line to separate historical and forecasted data
plt.axvline(x=last_date, color='k', linestyle='-', alpha=0.5)
plt.title('Stock Price Forecast for Next 30 Trading Days', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Price (INR)', fontsize=14)
plt.legend(loc='best', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()